# Jour 3 - Exercice 1 : Classification de la Satisfaction des Passagers

## Objectifs
- Appliquer les techniques de machine learning à un problème de classification réel
- Utiliser les données de satisfaction des passagers pour prédire leur niveau de satisfaction
- Évaluer les performances des modèles sur un jeu de test
- Visualiser et interpréter les résultats avec Plotly

## Introduction

Dans ce notebook, nous allons travailler sur un problème de classification binaire : prédire si un passager est satisfait ou non de son expérience de vol. Nous utiliserons les données de satisfaction des passagers et appliquerons différentes techniques de machine learning pour construire et évaluer des modèles de classification.

## 1. Chargement et exploration des données

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import pickle
import sys
import os

# Visualisation avec Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn pour le machine learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

# Modèles de classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

# Ajouter le chemin pour importer utils.py
sys.path.append(os.path.abspath('./'))
from utils import results_predictions_satisfaction

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement des données d'entraînement
train_df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
train_df.head()

In [ ]:
# Informations sur le DataFrame
train_df.info()

In [ ]:
# Statistiques descriptives
train_df.describe()

In [ ]:
# Vérification des valeurs manquantes
missing_values = train_df.isnull().sum()
missing_values_pct = (missing_values / len(train_df)) * 100

# Création d'un DataFrame pour visualiser les valeurs manquantes
missing_df = pd.DataFrame({
    'Nombre de valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_values_pct
})

# Affichage des colonnes avec des valeurs manquantes
missing_df[missing_df['Nombre de valeurs manquantes'] > 0].sort_values('Nombre de valeurs manquantes', ascending=False)

## 2. Visualisation des données

Explorons les données pour mieux comprendre la distribution des variables et leurs relations avec la satisfaction des passagers.

In [ ]:
# Distribution de la variable cible
satisfaction_counts = train_df['Satisfaction'].value_counts().reset_index()
satisfaction_counts.columns = ['Satisfaction', 'Count']

fig = px.pie(satisfaction_counts, values='Count', names='Satisfaction', 
             title='Distribution de la satisfaction des passagers',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
# Relation entre l'âge et la satisfaction
fig = px.histogram(train_df, x='Age', color='Satisfaction', 
                  title='Distribution de l\'âge par niveau de satisfaction',
                  barmode='group', nbins=20,
                  color_discrete_sequence=px.colors.qualitative.Set2)
fig.show()

In [ ]:
# Relation entre la classe et la satisfaction
class_satisfaction = pd.crosstab(train_df['Class'], train_df['Satisfaction'])
class_satisfaction_pct = class_satisfaction.div(class_satisfaction.sum(axis=1), axis=0) * 100

fig = px.bar(class_satisfaction_pct.reset_index().melt(id_vars='Class', var_name='Satisfaction', value_name='Percentage'),
            x='Class', y='Percentage', color='Satisfaction',
            title='Pourcentage de satisfaction par classe',
            color_discrete_sequence=px.colors.qualitative.Set2)
fig.show()

In [ ]:
# Relation entre le type de voyage et la satisfaction
travel_satisfaction = pd.crosstab(train_df['Type of Travel'], train_df['Satisfaction'])
travel_satisfaction_pct = travel_satisfaction.div(travel_satisfaction.sum(axis=1), axis=0) * 100

fig = px.bar(travel_satisfaction_pct.reset_index().melt(id_vars='Type of Travel', var_name='Satisfaction', value_name='Percentage'),
            x='Type of Travel', y='Percentage', color='Satisfaction',
            title='Pourcentage de satisfaction par type de voyage',
            color_discrete_sequence=px.colors.qualitative.Set2)
fig.show()

In [ ]:
# Visualisation des scores de satisfaction pour différents services
service_columns = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
                   'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
                   'Inflight entertainment', 'On-board service', 'Leg room service',
                   'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Créer un DataFrame pour la visualisation
service_data = []
for col in service_columns:
    for satisfaction in train_df['Satisfaction'].unique():
        subset = train_df[train_df['Satisfaction'] == satisfaction]
        for rating in range(6):  # 0-5 ratings
            count = (subset[col] == rating).sum()
            service_data.append({
                'Service': col,
                'Rating': rating,
                'Satisfaction': satisfaction,
                'Count': count
            })

service_df = pd.DataFrame(service_data)

# Créer un subplot pour chaque service
fig = make_subplots(rows=7, cols=2, subplot_titles=service_columns, vertical_spacing=0.1)

row, col = 1, 1
for service in service_columns:
    service_subset = service_df[service_df['Service'] == service]
    
    for i, satisfaction in enumerate(train_df['Satisfaction'].unique()):
        satisfaction_subset = service_subset[service_subset['Satisfaction'] == satisfaction]
        
        fig.add_trace(
            go.Bar(
                x=satisfaction_subset['Rating'],
                y=satisfaction_subset['Count'],
                name=satisfaction if row == 1 and col == 1 else None,
                showlegend=(row == 1 and col == 1),
                marker_color=px.colors.qualitative.Set2[i]
            ),
            row=row, col=col
        )
    
    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(height=1800, width=1000, title_text="Distribution des évaluations de service par niveau de satisfaction")
fig.show()

## 3. Préparation des données pour le modèle

Nous allons maintenant préparer les données pour l'entraînement du modèle.

In [ ]:
# Suppression de la colonne ID qui n'est pas utile pour la prédiction
train_df = train_df.drop('ID', axis=1)

# Séparation des features et de la variable cible
X = train_df.drop('Satisfaction', axis=1)
y = train_df['Satisfaction']

# Séparation des variables numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Variables numériques: {len(numeric_features)}")
print(f"Variables catégorielles: {len(categorical_features)}")

In [12]:
# Création d'un pipeline de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Séparation des données d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Entraînement et évaluation des modèles

Nous allons maintenant entraîner différents modèles de classification et évaluer leurs performances.

In [ ]:
# Définition des modèles à tester
models = {
    'Régression Logistique': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Dictionnaire pour stocker les résultats
results = {}

# Entraînement et évaluation de chaque modèle
for name, model in models.items():
    print(f"Entraînement du modèle: {name}")
    
    # Création du pipeline complet
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
    
    # Entraînement du modèle
    pipeline.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de validation
    y_pred = pipeline.predict(X_val)
    
    # Calcul des métriques
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, pos_label='satisfied')
    
    # Stockage des résultats
    results[name] = {
        'pipeline': pipeline,
        'accuracy': accuracy,
        'f1_score': f1
    }
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("\nRapport de classification:")
    print(classification_report(y_val, y_pred))
    print("\n" + "-"*50 + "\n")

In [ ]:
# Visualisation des résultats
model_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in model_names]
f1_scores = [results[name]['f1_score'] for name in model_names]

fig = go.Figure(data=[
    go.Bar(name='Accuracy', x=model_names, y=accuracies, marker_color='royalblue'),
    go.Bar(name='F1-score', x=model_names, y=f1_scores, marker_color='lightcoral')
])

fig.update_layout(
    title='Comparaison des performances des modèles',
    xaxis_title='Modèle',
    yaxis_title='Score',
    barmode='group',
    yaxis=dict(range=[0, 1])
)

fig.show()

## 5. Évaluation sur le jeu de test

Maintenant, nous allons évaluer notre meilleur modèle sur le jeu de test.

In [ ]:
# Chargement des données de test
test_df = pd.read_csv('../../data/passenger_satisfaction/test.csv')
test_df = test_df.drop('ID', axis=1)

# Affichage des premières lignes
test_df.head()

In [ ]:
# Sélection du meilleur modèle (celui avec le meilleur F1-score)
best_model_name = max(results, key=lambda x: results[x]['f1_score'])
best_pipeline = results[best_model_name]['pipeline']

print(f"Meilleur modèle: {best_model_name}")
print(f"F1-score sur la validation: {results[best_model_name]['f1_score']:.4f}")

In [ ]:
# Prédictions sur le jeu de test
test_predictions = best_pipeline.predict(test_df)

# Affichage des premières prédictions
pd.DataFrame({
    'Prédiction': test_predictions
}).head(10)

In [ ]:
# Évaluation des prédictions avec la fonction results_predictions_satisfaction
test_f1_score = results_predictions_satisfaction(test_predictions)
print(f"F1-score sur le jeu de test: {test_f1_score:.4f}")

## 6. Analyse des caractéristiques importantes (pour Random Forest)

Si nous avons utilisé un modèle Random Forest, nous pouvons analyser l'importance des caractéristiques.

In [ ]:
# Vérification si le meilleur modèle est un Random Forest
if best_model_name == 'Random Forest':
    # Extraction du modèle Random Forest du pipeline
    rf_model = best_pipeline.named_steps['classifier']
    
    # Extraction des noms des caractéristiques après prétraitement
    preprocessor = best_pipeline.named_steps['preprocessor']
    cat_features = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features)
    feature_names = numeric_features + list(cat_features)
    
    # Extraction des importances des caractéristiques
    importances = rf_model.feature_importances_
    
    # Création d'un DataFrame pour la visualisation
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False).head(20)
    
    # Visualisation
    fig = px.bar(feature_importance_df, x='Importance', y='Feature', 
                orientation='h', title='Top 20 des caractéristiques les plus importantes',
                color='Importance', color_continuous_scale='Viridis')
    fig.show()
else:
    print(f"Le meilleur modèle est {best_model_name}, qui ne fournit pas d'importance des caractéristiques.")

## 7. Conclusion

Dans ce notebook, nous avons:
1. Exploré et visualisé les données de satisfaction des passagers
2. Préparé les données pour l'entraînement des modèles
3. Entraîné et évalué plusieurs modèles de classification
4. Sélectionné le meilleur modèle et évalué ses performances sur le jeu de test
5. Analysé les caractéristiques les plus importantes (si applicable)

Le modèle final a obtenu un F1-score de [valeur] sur le jeu de test, ce qui indique [interprétation des performances].

### Pistes d'amélioration
- Essayer d'autres algorithmes de classification (XGBoost, LightGBM, etc.)
- Optimiser les hyperparamètres des modèles avec une recherche par grille ou une recherche aléatoire
- Créer de nouvelles caractéristiques à partir des données existantes
- Utiliser des techniques d'ensemble pour combiner plusieurs modèles